In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df = pd.read_csv(r"C:\Users\Asus\OneDrive\Desktop\Dataset\train_titanic_dataset.csv")[['Age','Fare','Survived','Parch','SibSp']]
df

,Age,Fare,Survived,Parch,SibSp
0,22.0,7.2500,0,0,1
1,38.0,71.2833,1,0,1
2,26.0,7.9250,1,0,0
3,35.0,53.1000,1,0,1
4,35.0,8.0500,0,0,0
...,...,...,...,...,...
886,27.0,13.0000,0,0,0
887,19.0,30.0000,1,0,0
888,NaN,23.4500,0,2,1
889,26.0,30.0000,1,0,0


In [5]:
df.dropna(inplace = True)

In [8]:
df.isnull().sum()

Age         0
Fare        0
Survived    0
Parch       0
SibSp       0
dtype: int64

In [9]:
# mai jod deta hu sibling spouse aur parent child
df['family'] = df['SibSp']+df['Parch']

In [10]:
df.head()

,Age,Fare,Survived,Parch,SibSp,family
0,22.0,7.2500,0,0,1,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,1,0,0,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [13]:
df.drop(columns=["Parch","SibSp"],inplace = True)

In [14]:
df.head()

,Age,Fare,Survived,family
0,22.0,7.2500,0,1
1,38.0,71.2833,1,1
2,26.0,7.9250,1,0
3,35.0,53.1000,1,1
4,35.0,8.0500,0,0


In [15]:
x = df.drop(columns = ['Survived'])
y = df['Survived']

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2,random_state = 42)

In [22]:
x_train

,Age,Fare,family
328,31.0,20.5250,2
73,26.0,14.4542,1
253,30.0,16.1000,1
719,33.0,7.7750,0
666,25.0,13.0000,0
...,...,...,...
92,46.0,61.1750,1
134,25.0,13.0000,0
337,41.0,134.5000,0
548,33.0,20.5250,2


In [17]:
# without binarization
clf = DecisionTreeClassifier()


In [19]:
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [20]:
y_pred = clf.predict(x_test)

In [21]:
accuracy_score(y_test,y_pred)

0.6083916083916084

In [23]:
# cross validatioon
cross_val_score(clf,x,y,scoring = 'accuracy',cv=10)

array([0.65277778, 0.61111111, 0.625     , 0.65277778, 0.57746479,
       0.66197183, 0.73239437, 0.6056338 , 0.64788732, 0.63380282])

In [24]:
np.mean(cross_val_score(clf,x,y,scoring = 'accuracy',cv=10))

0.6457159624413146

### Binarization ka concept kaisa lgaya ki agar koe banda aklee travel kar rha hai toh faily ki value 
### 0 hai but agar family ki value agar 0 se jayda hau matlab family ke stah tarvael kar rha hai.


# applying Biranizer


In [12]:
df.head()

,Age,Fare,Survived,Parch,SibSp,family
0,22.0,7.2500,0,0,1,1
1,38.0,71.2833,1,0,1,1
2,26.0,7.9250,1,0,0,0
3,35.0,53.1000,1,0,1,1
4,35.0,8.0500,0,0,0,0


In [25]:
from sklearn.preprocessing import Binarizer

In [30]:
trf = ColumnTransformer([
    ('bin',Binarizer(copy = False),['family'])],# yha code pe dhyan dena list yhi par khatam ho gaayi
remainder = 'passthrough')

# copy = False rkha hai kyoki existing hi column me changes karna hai.

In [31]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf =  trf.transform(x_test)

In [32]:
x_train_trf

array([[  1.    ,  31.    ,  20.525 ],
       [  1.    ,  26.    ,  14.4542],
       [  1.    ,  30.    ,  16.1   ],
       ...,
       [  0.    ,  41.    , 134.5   ],
       [  1.    ,  33.    ,  20.525 ],
       [  0.    ,  33.    ,   7.8958]])

In [33]:
pd.DataFrame(x_train_trf,columns = ['Age','Fare','Survived'])

,Age,Fare,Survived
0,1.0,31.0,20.5250
1,1.0,26.0,14.4542
2,1.0,30.0,16.1000
3,0.0,33.0,7.7750
4,0.0,25.0,13.0000
...,...,...,...
566,1.0,46.0,61.1750
567,0.0,25.0,13.0000
568,0.0,41.0,134.5000
569,1.0,33.0,20.5250


In [34]:
# DECISION TREE ALGORTHIM LGA DIYA
clf = DecisionTreeClassifier()


In [36]:
clf.fit(x_train_trf,y_train)


DecisionTreeClassifier()

In [37]:
y_pred2 = clf.predict(x_test_trf)



In [40]:
accuracy_score(y_test,y_pred2)
# accuracy improve kar gayi

0.6223776223776224

In [41]:
# cross_val_score
x_trf = trf.fit_transform(x)

In [42]:
cross_val_score(DecisionTreeClassifier(),x_trf,y,cv=10,scoring = 'accuracy')

array([0.59722222, 0.58333333, 0.59722222, 0.63888889, 0.5915493 ,
       0.70422535, 0.71830986, 0.61971831, 0.56338028, 0.63380282])

In [43]:
np.mean(cross_val_score(DecisionTreeClassifier(),x_trf,y,cv=10,scoring = 'accuracy'))
# it near about

0.6247652582159624